# TD : Visualisation de données avec Bokeh

Dans ce tutoriel, nous allons explorer la bibliothèque [**Bokeh**](https://docs.bokeh.org/en/latest/) pour réaliser des visualisations interactives en Python. Contrairement à des bibliothèques comme Matplotlib ou Seaborn, Bokeh permet d'obtenir des **graphiques dynamiques** et **interactifs** (zoom, survol, etc.) directement depuis un notebook Jupyter, ou exportés vers des pages web autonomes.

Nous allons illustrer les principales fonctionnalités de Bokeh à l'aide de deux datasets :

1. Le dataset **tips** (issu de Seaborn), qui contient des informations sur des pourboires au restaurant.
2. Le dataset **flights** (issu également de Seaborn), qui contient des informations sur le nombre de passagers aériens mois par mois.

Enfin, nous verrons comment créer des **visualisations dynamiques** avec des widgets (sliders, menus).

## Objectifs

1. Découvrir les bases de Bokeh : `figure`, `show`, `ColumnDataSource`, etc.
2. Créer différents graphiques avec Bokeh : scatter, line, bar, histogrammes, heatmap simplifiée, etc.
3. Utiliser les outils interactifs : zoom, pan, hover, légendes dynamiques.
4. Créer des graphiques dynamiques avec des widgets (Sliders, Select) et des callbacks.
5. Visualiser des données géographiques (cartes) via les **tile providers** (ex. CartoDB) et centrer la vue sur la Mauritanie.

---

## 1. Installation et importations

Si vous n'avez pas encore Bokeh installé, exécutez la commande suivante dans une cellule ou un terminal :

```bash
!pip install bokeh
```

Ensuite, importons les modules nécessaires :

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar
from bokeh.models import Slider, Select, CustomJS
from bokeh.layouts import row, column
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256, Category10
#from bokeh.models.tiles import CARTODBPOSITRON

# Pour afficher les figures Bokeh directement dans le notebook
output_notebook()

# Chargement du dataset tips
tips = sns.load_dataset("tips")
tips.head()

Loading BokehJS ...

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Aperçu du dataset **tips**
- **total_bill** : le montant total de l'addition
- **tip** : le montant du pourboire
- **sex** : le sexe de la personne payant
- **smoker** : si la personne est fumeuse ou non
- **day** : le jour de la semaine (Thur, Fri, Sat, Sun)
- **time** : le moment du service (Lunch, Dinner)
- **size** : le nombre de personnes à la table

Nous allons également utiliser le dataset **flights** pour la partie séries temporelles :

In [ ]:
flights = sns.load_dataset("flights")
flights.head()

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121


Le dataset **flights** contient :
- **year** : l'année (entre 1949 et 1960)
- **month** : le mois (Jan, Feb, etc.)
- **passengers** : le nombre de passagers transportés

---
## 2. Bases de Bokeh : figure et ColumnDataSource

Pour créer un graphique avec Bokeh :
1. On définit une `figure` (la zone de dessin).
2. On ajoute une ou plusieurs « glyphs » (formes graphiques) : `circle`, `line`, `vbar`, etc.
3. On utilise `show()` pour l'afficher.

Bokeh utilise souvent un objet `ColumnDataSource` (CDS) pour stocker les données. Cela facilite la création de graphiques interactifs et la mise à jour.

### Exemple simple : scatterplot (nuage de points)
Visualisons la relation entre `total_bill` et `tip` du dataset **tips**.

In [ ]:
# Préparation des données dans un ColumnDataSource
source_tips = ColumnDataSource(tips)

# Création d'une figure
p_scatter = figure(
    width=600,
    height=400,
    title="Relation entre total_bill et tip (Bokeh)",
    x_axis_label="total_bill",
    y_axis_label="tip",
    tools=["pan", "wheel_zoom", "box_zoom", "reset", "hover"]
)

# Ajout d'un glyph circle
p_scatter.scatter(
    x="total_bill",
    y="tip",
    source=source_tips,
    size=8,
    color="navy",
    alpha=0.6,
    legend_label="Pourboires"
)

# Personnalisation de la légende
p_scatter.legend.location = "top_left"

# Pour permettre de voir des infos au survol (hover)
hover_tool = p_scatter.select_one(HoverTool)
hover_tool.tooltips = [
    ("tip", "@tip"),
    ("day", "@day"),
]

show(p_scatter)

Nous disposons maintenant d'un scatterplot interactif : vous pouvez zoomer, déplacer, réinitialiser l'affichage et, au survol, afficher des informations supplémentaires.

---
## 3. Différents types de graphiques avec Bokeh

### 3.1 Graphique en ligne (Lineplot) : Série temporelle

Reprenons le dataset **flights** qui contient une série mensuelle de passagers. Nous allons créer un **lineplot** afin de visualiser l'évolution du trafic passagers au cours du temps.

In [ ]:
# On crée une colonne date (année-mois) pour avoir un vrai index temporel.
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

flights['month_num'] = flights['month'].map(month_map)
flights['date'] = pd.to_datetime(
    {
        'year': flights['year'],
        'month': flights['month_num'],
        'day': 1
    }
)

# Tri des données par date (important pour un tracé propre)
flights_sorted = flights.sort_values(by="date")

# Conversion en ColumnDataSource
source_flights = ColumnDataSource(flights_sorted)

# Création de la figure
p_line = figure(
    width=700,
    height=400,
    x_axis_type="datetime",
    title="Évolution du trafic passagers (1949-1960)",
    tools=["pan", "wheel_zoom", "box_zoom", "reset", "hover", "save"]
)

# Tracé d'une ligne
p_line.line(
    x="date",
    y="passengers",
    source=source_flights,
    color="green",
    line_width=2,
    legend_label="Nombre de passagers"
)

# Ajout de points pour rendre l'évolution plus visible
p_line.scatter(
    x="date",
    y="passengers",
    source=source_flights,
    color="green",
    size=6,
    alpha=0.8
)

# Configuration de la légende et du hover
p_line.legend.location = "top_left"
hover_tool_line = p_line.select_one(HoverTool)
hover_tool_line.tooltips = [
    ("Date", "@date{%F}"),  # %F => YYYY-MM-DD
    ("Passagers", "@passengers")
]
hover_tool_line.formatters = {
    '@date': 'datetime'
}

show(p_line)

### 3.2 Barplot

Nous pouvons utiliser un barplot pour représenter, par exemple, la moyenne de `total_bill` ou de `tip` en fonction du jour. Comme Bokeh ne fournit pas de fonction de barplot « haut niveau » (comme Seaborn), on utilise la fonction `vbar` ou `hbar` d'une `figure`.

In [ ]:
# Exemple : barplot de la moyenne de total_bill par jour

# On calcule la moyenne par jour
mean_by_day = tips.groupby("day", observed=True)[["total_bill"]].mean().reset_index()
mean_by_day.columns = ["day", "mean_bill"]

# Pour l'ordre des catégories (Thur, Fri, Sat, Sun)
days_order = ["Thur", "Fri", "Sat", "Sun"]

# Convertissons en ColumnDataSource
source_mean = ColumnDataSource(mean_by_day)

# Création de la figure
p_bar = figure(
    y_range=days_order,
    width=500,
    height=400,
    title="Moyenne de l'addition par jour",
    tools=["pan", "wheel_zoom", "box_zoom", "reset", "hover", "save"]
)

# Ajout d'un barplot
p_bar.hbar(
    y="day",
    right="mean_bill",
    height=0.6,
    source=source_mean,
    color="blue",
    alpha=0.7,
    legend_label="total_bill"
)

# Personnalisation
p_bar.xaxis.axis_label = "Jour"
p_bar.yaxis.axis_label = "Montant moyen"
p_bar.legend.location = "top_left"

# Hover tool
p_bar.select_one(HoverTool).tooltips = [
    ("Jour", "@day"),
    ("Moyenne", "@mean_bill")
]

show(p_bar)

### 3.3 Histogramme (displot-like)

Pour réaliser un histogramme en Bokeh, nous devons :

1. Calculer les *bins* (classes) pour nos données.
2. Calculer l'histogramme (via `np.histogram` par exemple).
3. Tracer ces données via `quad`, `vbar`, etc.

Exemple avec la distribution de `total_bill`.

In [ ]:
hist, edges = np.histogram(tips["total_bill"], bins=10)

# Conversion en ColumnDataSource
hist_source = ColumnDataSource(dict(
    top=hist,
    left=edges[:-1],
    right=edges[1:]
))

p_hist = figure(
    width=500,
    height=400,
    title="Distribution de total_bill",
    tools=["pan", "wheel_zoom", "box_zoom", "reset", "save"]
)

# Tracé de l'histogramme
p_hist.quad(
    top="top",
    bottom=0,
    left="left",
    right="right",
    source=hist_source,
    fill_color="orange",
    line_color="black",
    alpha=0.7
)

p_hist.xaxis.axis_label = "total_bill"
p_hist.yaxis.axis_label = "Fréquence"

show(p_hist)

### 3.4 Heatmap simplifiée : corrélation

Pour visualiser les corrélations entre variables numériques du dataset **tips**, nous devons :

1. Conserver uniquement les colonnes numériques pour éviter les erreurs (strings -> float).
2. Calculer la matrice de corrélation.
3. Tracer une "heatmap" (on peut utiliser un `rect` ou `image` glyph dans Bokeh).

Nous allons ici faire un exemple simplifié en utilisant un `rect` par case :

In [ ]:
# Sélection des colonnes numériques
numerical_tips = tips.select_dtypes(include='number')
corr_matrix = numerical_tips.corr()

# On convertit la matrice en "long format"
corr_df = corr_matrix.stack().reset_index()
corr_df.columns = ["var1", "var2", "correlation"]

# On récupère l'ordre des variables
vars_order = list(numerical_tips.columns)

# ColumnDataSource
source_corr = ColumnDataSource(corr_df)

# Création de la figure
p_heat = figure(
    x_range=vars_order,
    y_range=list(reversed(vars_order)),
    width=900,
    height=800,
    title="Matrice de corrélation",
    tools=["hover", "save"],
    toolbar_location="right"
)

# Mappage de couleur (de -1 à 1)
color_mapper = LinearColorMapper(palette=Viridis256, low=-1, high=1)

# Ajout des rectangles
p_heat.rect(
    x="var1",
    y="var2",
    width=1,
    height=1,
    source=source_corr,
    line_color=None,
    fill_color={"field": "correlation", "transform": color_mapper}
)

# Configuration du hover
p_heat.select_one(HoverTool).tooltips = [
    ("Variables", "@var1 vs @var2"),
    ("Corrélation", "@correlation{0.2f}")
]

# Inversion de l'axe Y pour correspondre à l'ordre habituel
p_heat.y_range.range_padding = 0

# Ajout d'une barre de couleurs
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, location=(0,0))
p_heat.add_layout(color_bar, 'right')

p_heat.xaxis.axis_label = "Variables"
p_heat.yaxis.axis_label = "Variables"

show(p_heat)

---
## 4. Graphiques dynamiques : Widgets et callbacks

Bokeh permet de créer des visualisations dynamiques en associant des widgets (slideurs, menus déroulants) à des callbacks JavaScript ou Python (via Bokeh Server). Nous ferons un court exemple avec un `Slider` qui filtre la colonne `tip` du dataset **tips**.

**Principe :**

1. On crée un `Slider` (par exemple, pour définir un seuil min de pourboire).
2. On associe un `CustomJS` callback qui met à jour la `ColumnDataSource` en fonction de la valeur du slider.


In [ ]:
# Copie du dataset 'tips' pour filtrage
source_original = ColumnDataSource(tips)
source_filtered = ColumnDataSource(tips)

# Figure
p_dynamic = figure(
    width=700,
    height=400,
    title="Filtre dynamique sur 'tip'",
    x_axis_label="total_bill",
    y_axis_label="tip",
    tools=["pan", "wheel_zoom", "box_zoom", "reset", "hover", "save"]
)

p_dynamic.scatter(
    x="total_bill",
    y="tip",
    source=source_filtered,
    size=8,
    color="red",
    alpha=0.6
)

# Slider
slider = Slider(start=0, end=10, value=0, step=0.5, title="Min tip")

# Callback JS : on filtre le dataset pour ne garder que les lignes où tip >= slider.value
callback_code = """
    const data_original = source_original.data;
    const data_filtered = source_filtered.data;
    const tip_min = cb_obj.value;

    const total_bill_orig = data_original['total_bill'];
    const tip_orig = data_original['tip'];
    const day_orig = data_original['day'];
    const size_orig = data_original['size'];

    let total_bill_f = [];
    let tip_f = [];
    let day_f = [];
    let size_f = [];

    for (let i = 0; i < tip_orig.length; i++) {
        if (tip_orig[i] >= tip_min) {
            total_bill_f.push(total_bill_orig[i]);
            tip_f.push(tip_orig[i]);
            day_f.push(day_orig[i]);
            size_f.push(size_orig[i]);
        }
    }

    data_filtered['total_bill'] = total_bill_f;
    data_filtered['tip'] = tip_f;
    data_filtered['day'] = day_f;
    data_filtered['size'] = size_f;

    source_filtered.change.emit();
"""

callback = CustomJS(
    args=dict(
        source_original=source_original,
        source_filtered=source_filtered
    ),
    code=callback_code
)

slider.js_on_change('value', callback)

# Assemblage dans une layout
layout_dynamic = column(slider, p_dynamic)
show(layout_dynamic)

En bougeant le slider, vous voyez les points disparaître ou réapparaître en fonction de la valeur minimale de `tip` choisie.

---
## 5. Cartes et visualisation géographique (centré sur la Mauritanie)

Bokeh propose des **tile providers** pour afficher des fonds de carte (OpenStreetMap, CartoDB, etc.). Vous pouvez ensuite y superposer vos propres données (longitude, latitude). Notez que Bokeh attend souvent des coordonnées projetées en Web Mercator (EPSG:3857).

### Exemple : Carte centrée sur la Mauritanie

Nous choisissons ici un *range* de coordonnées Web Mercator qui permettent d'afficher la Mauritanie sur la carte. Les valeurs `x_range` et `y_range` sont approximatives, mais suffisent à donner un aperçu.

Si vous avez des points GPS (lat, lon) pour la Mauritanie, vous devriez les convertir en coordonnées Web Mercator avant de les ajouter via `circle`, `triangle`, etc.

In [ ]:
# Création de la figure en mode "mercator"
p_map = figure(
    width=800,
    height=800,
    x_range=(-2200000, -600000),  # Valeurs approx. pour centrer sur la Mauritanie
    y_range=(1400000, 2900000),
    x_axis_type="mercator",
    y_axis_type="mercator",
    title="Exemple de carte Bokeh centrée sur la Mauritanie"
)

# On ajoute le fond de carte (OSM)
p_map.add_tile('OSM')

show(p_map)

Vous voyez maintenant un fond de carte orienté sur la Mauritanie. Vous pouvez zoomer, vous déplacer, etc. Pour superposer des points, il suffit de convertir leurs coordonnées GPS (lat, lon) en projection Web Mercator, puis d'utiliser `p_map.circle(...)`, `p_map.triangle(...)`, etc.

## Ressources supplémentaires

- [Documentation officielle de Bokeh](https://docs.bokeh.org/en/latest/)
- [Exemples Bokeh](https://docs.bokeh.org/en/latest/docs/gallery.html)
- [User Guide Bokeh](https://docs.bokeh.org/en/latest/docs/user_guide.html)

Avec Bokeh, vous pouvez intégrer vos visualisations dans des pages web, construire des applications de data exploration, et profiter d'interactions riches !

Bon apprentissage et bonne visualisation !